# Warning
This file is currently a work in progress. Expect instability, inconsistency, and ugly code.

In [52]:
# Constants
DATA_DIRECTORY = "../../data"

# Delete me, eventually
data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'

In [139]:
# Initial object constructors
class Workspace():
    def __init__(self, tree):
        # assert tree is array
        self.tree = tree
        
    # TODO: Fill out body when we need the workspace   
    def update(self, action):
        if action.what == 'INIT':
            return Workspace(['root'])
        elif action.what == 'MSG':
            return self
        elif action.what == 'ADD':
            return self
    
class Action():
    def __init__(self, who, what, content=False):
        self.who = who
        self.what = what
        self.content = content
        
    def to_string(self):
        if self.content:
            return f'{self.who}: {self.what} {self.content}'
        else:
            return f'{self.who}: {self.what}'

In [140]:
def string_to_action(s):
    who = s[0]
    i = s.index(" ")
    line = s[i+1:]
    j = line.index(" ")
    what = line[0:j]
    content = line[j+1:]
    return Action(who, what, content)
    

In [154]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    
    action_list = [init_entry]
    current_workspace = init_entry["workspace"]
    
    for line in open(filename, 'r'):
        line = line[:-1]
        if not line:
            continue
        try: 
            action_list.append({"workspace": current_workspace,
                                "action": string_to_action(line)})
        except ValueError:
            print("This shouldn't happen!")
            return
    
    return action_list
   

In [155]:
# Todo
def files_to_words(filenames):
    return None

In [158]:
def load_babi_data():
    data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
    training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
    dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'
    types_and_tokens = [
            { "type": 'cuisine', "tokens": ['french', 'italian', 'british', 'spanish', 'indian'] },
            { "type": 'location', "tokens": ['rome', 'london', 'bombay', 'paris', 'madrid'] },
            { "type": 'price', "tokens": ['cheap', 'moderate', 'expensive'] },
            { "type": 'people', "tokens": ['two', 'four', 'six', 'eight'] },
        ]
    
    data = {
      "training": file_to_dialogs(training_file),
      "dev": file_to_dialogs(dev_file),
      #TODO: words: filesToWords([trainingFile]).concat(specialWords),
      #"typesAndTokens"
    }

    return data

In [159]:
load_babi_data()

{'dev': [{'action': <__main__.Action at 0x7f9faf71fda0>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76fc50>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76fcc0>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76fd30>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76fda0>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76fe48>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76feb8>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76ff28>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action at 0x7f9faf76ff98>,
   'workspace': <__main__.Workspace at 0x7f9faf71fdd8>},
  {'action': <__main__.Action 

# WARNING!
Everything below this cell is currently non-functional. If it compiles, I'll be rather surprised.

In [ ]:
import torch

# Adapted from:
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# TODO: Implement make OneHot
class Encoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, x, h):
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h

# TODO: Implement initialization


# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

        self.out = nn.Linear(h_size, )
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = T.softmax(torch.Tensor.map_(encoder_states, zero_states,
                                                      combinedState))

        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h